In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from itertools import combinations
from datetime import datetime
import pandas as pd
import numpy as np
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.regression.linear_model import OLS

In [2]:
table2_data_raw = pd.read_csv("data/merged_data.csv")
# only need the rows with the data, first 4 rows and last 1 row are not needed
table2_ff5 = pd.read_csv("data/FF5_monthly.csv")
table2_UMD = pd.read_csv("data/FF_Momentum_monthly.csv")
# Rename columns to ensure 'date' column is correctly named

table2_ff5.columns = ['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
table2_UMD.columns = ['date', 'UMD']

table2_data_raw['date'] = pd.to_datetime(table2_data_raw['date'])
table2_ff5['date'] = pd.to_datetime(table2_ff5['date'], format='%Y%m')
table2_UMD['date'] = pd.to_datetime(table2_UMD['date'], format='%Y%m')

In [3]:
# Get the data original sample reseatch, so first we need to get the data before 1963-07-01 to ensure 
start_date_original = '1963-07-01'
end_date = '2023-12-29' #20231230 and 20231231 are weekends, not in the data
print(f'Predictor number of start month: {len(table2_data_raw[table2_data_raw['date'] < start_date_original]['predictor'].unique())}') # check if the data aviablable for the whole period
factor_original_data = table2_data_raw[table2_data_raw['date'] < start_date_original]['predictor'].unique()
table2_data_original = table2_data_raw[table2_data_raw['predictor'].isin(factor_original_data)]
table2_data_original = table2_data_original[(table2_data_original['date'] >= start_date_original) & (table2_data_original['date'] <= end_date)]
print(f'Predictor number of end month: {len(table2_data_original[table2_data_original['date'] == end_date]['predictor'].unique())}') #check if the data aviablable for the whole period
table2_data_original

Predictor number of start month: 50
Predictor number of end month: 50


,date,port01,port02,port03,port04,port05,portLS,predictor
2767,1963-07-01,-0.694908,-0.449173,-0.394214,-0.721289,-0.849711,-0.154803,Accruals
2768,1963-07-02,0.802712,0.600216,0.585548,1.000894,0.920318,0.117607,Accruals
2769,1963-07-03,0.692565,0.383415,0.442535,0.792983,0.736245,0.043681,Accruals
2770,1963-07-05,0.134336,0.265622,0.490242,0.298404,0.659313,0.524977,Accruals
2771,1963-07-08,-0.486027,-0.143149,-0.228552,-0.508734,-1.130730,-0.644703,Accruals
...,...,...,...,...,...,...,...,...
1237143,2023-12-22,1.857859,0.723824,0.498934,0.233261,0.428393,-1.429466,std_turn
1237144,2023-12-26,3.266919,0.875071,1.303056,0.540725,0.561904,-2.705015,std_turn
1237145,2023-12-27,2.341186,0.200744,0.285204,0.159429,0.059149,-2.282037,std_turn
1237146,2023-12-28,-1.669881,-0.263774,-0.543158,-0.266308,-0.110725,1.559157,std_turn


In [4]:
# Update data from 1999-01-01 to 2023-12-29, 
start_date_update = '1999-01-01'
end_date = '2023-12-29'
table2_data = table2_data_raw[(table2_data_raw['date'] >= start_date_update) & (table2_data_raw['date'] <= end_date)].copy()
# Get the remaining factors belonging to the time frame
unique_predictors_update = table2_data['predictor'].unique()
print(f'The list of predictors is: \n{unique_predictors_update}')
print(f'Total predictor number is: {len(unique_predictors_update)}')

The list of predictors is: 
['Accruals' 'AnalystValue' 'AssetGrowth' 'BM' 'BPEBM' 'Beta'
 'BetaLiquidityPS' 'BookLeverage' 'CBOperProf' 'CF' 'CPVolSpread'
 'ChAssetTurnover' 'ChNWC' 'CompEquIss' 'CompositeDebtIssuance'
 'Coskewness' 'CustomerMomentum' 'DolVol' 'EBM' 'EP' 'EarningsSurprise'
 'FirmAge' 'Frontier' 'GP' 'Herf' 'High52' 'IdioVol3F' 'Illiquidity'
 'IntMom' 'InvGrowth' 'LRreversal' 'MaxRet' 'Mom12m' 'Mom6m' 'Mom6mJunk'
 'MomOffSeason' 'MomOffSeason06YrPlus' 'MomOffSeason11YrPlus'
 'MomOffSeason16YrPlus' 'MomSeason' 'MomSeason06YrPlus'
 'MomSeason11YrPlus' 'MomSeason16YrPlus' 'MomSeasonShort' 'NOA' 'OperProf'
 'PS' 'RDAbility' 'RIVolSpread' 'ResidualMomentum' 'RoE' 'SP' 'STreversal'
 'ShareIss1Y' 'ShareIss5Y' 'Size' 'VolMkt' 'VolSD' 'XFIN' 'cfp' 'roaq'
 'std_turn']
Total predictor number is: 62


In [5]:
# first get the monthly returns for each stock, sum of the daily returns by month
table2_data['month'] = table2_data['date'].dt.month
table2_data['year'] = table2_data['date'].dt.year
table2_data['month_year'] = table2_data['date'].dt.to_period('M')
# sum the daily returns by month
table2_data_monthly = table2_data.groupby(['month_year', 'predictor'])[['port01','port02','port03','port04','port05','portLS']].sum().reset_index()
table2_data_monthly['date'] = table2_data_monthly['month_year'].dt.to_timestamp()
table2_data_monthly = table2_data_monthly.drop(['month_year'], axis=1)
table2_data_monthly = table2_data_monthly.sort_values(by=['predictor','date'])
table2_data_monthly = table2_data_monthly.reset_index(drop=True)
# put date in first column
cols = table2_data_monthly.columns.tolist()
cols = cols[-1:] + cols[:-1]
table2_data_monthly = table2_data_monthly[cols]
table2_data_monthly.set_index('date', inplace=True)
# divide 100
table2_data_monthly[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] = table2_data_monthly[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] / 100
table2_data_monthly['portLS2'] = table2_data_monthly['port05'] + table2_data_monthly['port04'] - table2_data_monthly['port02'] - table2_data_monthly['port01']
table2_data_monthly

,predictor,port01,port02,port03,port04,port05,portLS,portLS2
date,,,,,,,,
1999-01-01,Accruals,0.079906,-0.001093,0.026478,0.005808,0.081018,0.001112,0.008013
1999-02-01,Accruals,-0.049856,-0.016759,-0.012612,-0.026950,-0.072324,-0.022469,-0.032660
1999-03-01,Accruals,0.043410,0.037410,0.034296,0.036752,0.041223,-0.002187,-0.002844
1999-04-01,Accruals,0.067969,0.022862,0.022378,0.077499,0.028817,-0.039152,0.015485
1999-05-01,Accruals,-0.029193,-0.018245,-0.017828,0.001053,-0.024211,0.004982,0.024280
...,...,...,...,...,...,...,...,...
2023-08-01,std_turn,-0.188815,-0.068048,-0.025704,-0.032045,-0.022582,0.166233,0.202236
2023-09-01,std_turn,-0.067067,-0.050836,-0.034021,-0.052222,-0.044330,0.022738,0.021352
2023-10-01,std_turn,-0.138222,-0.100143,-0.061419,-0.033710,-0.044634,0.093587,0.160021


In [6]:
# modify the table2_data_original like table2_data above
table2_data_original['month'] = table2_data_original['date'].dt.month
table2_data_original['year'] = table2_data_original['date'].dt.year
table2_data_original['month_year'] = table2_data_original['date'].dt.to_period('M')
# sum the daily returns by month
table2_data_monthly_original = table2_data_original.groupby(['month_year', 'predictor'])[['port01','port02','port03','port04','port05','portLS']].sum().reset_index()
table2_data_monthly_original['date'] = table2_data_monthly_original['month_year'].dt.to_timestamp()
table2_data_monthly_original = table2_data_monthly_original.drop(['month_year'], axis=1)
table2_data_monthly_original = table2_data_monthly_original.sort_values(by=['predictor','date'])
table2_data_monthly_original = table2_data_monthly_original.reset_index(drop=True)
# put date in first column
cols = table2_data_monthly_original.columns.tolist()
cols = cols[-1:] + cols[:-1]
table2_data_monthly_original = table2_data_monthly_original[cols]
table2_data_monthly_original.set_index('date', inplace=True)
# divide 100
table2_data_monthly_original[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] = table2_data_monthly_original[['port01', 'port02', 'port03', 'port04', 'port05', 'portLS']] / 100
table2_data_monthly_original['portLS2'] = table2_data_monthly_original['port05'] + table2_data_monthly_original['port04'] - table2_data_monthly_original['port02'] - table2_data_monthly_original['port01']
table2_data_monthly_original


,predictor,port01,port02,port03,port04,port05,portLS,portLS2
date,,,,,,,,
1963-07-01,Accruals,-0.022245,0.007165,-0.002217,0.003021,0.003446,0.025691,0.021546
1963-08-01,Accruals,0.056922,0.053736,0.059434,0.048730,0.047843,-0.009079,-0.014086
1963-09-01,Accruals,-0.010046,-0.014426,-0.025168,-0.002418,-0.008202,0.001844,0.013852
1963-10-01,Accruals,0.014035,0.020162,0.013247,0.027338,0.058784,0.044749,0.051925
1963-11-01,Accruals,-0.005624,-0.008867,-0.007389,0.016824,-0.020129,-0.014506,0.011186
...,...,...,...,...,...,...,...,...
2023-08-01,std_turn,-0.188815,-0.068048,-0.025704,-0.032045,-0.022582,0.166233,0.202236
2023-09-01,std_turn,-0.067067,-0.050836,-0.034021,-0.052222,-0.044330,0.022738,0.021352
2023-10-01,std_turn,-0.138222,-0.100143,-0.061419,-0.033710,-0.044634,0.093587,0.160021


## Table 2
### Panel A

In [7]:
# Define Factor Momentum Strategy Function
def factor_momentum_strategy(returns, L, H, n_long_short):

    strategy_returns = pd.Series(index=returns.index, dtype=float)
    N = returns.shape[1]  
    for t in range(L, len(returns)):
        # Formation period: t-L to t-1 (using full data history up to t)
        past_returns = returns.iloc[t-L:t].mean()  # Average return over last L months
        # Rank factors
        ranked_factors = past_returns.sort_values(ascending=False)
        # Select top and bottom n_long_short factors
        long_factors = ranked_factors.index[:n_long_short]
        short_factors = ranked_factors.index[-n_long_short:]
        
        # Holding period: t to t+H-1
        if H == 1:
            # For H=1, take the next month's return
            long_returns = returns.iloc[t][long_factors].mean()
            short_returns = returns.iloc[t][short_factors].mean()
            strategy_returns.iloc[t] = long_returns - short_returns
        else:
            holding_returns = []
            for h in range(H):
                if t - h >= L:  # to ensure the data have enough history L months
                    past_h = returns.iloc[t-h-L:t-h].mean()  # to calculate the average return over last L months
                    ranked_h = past_h.sort_values(ascending=False)
                    long_h = ranked_h.index[:n_long_short]
                    short_h = ranked_h.index[-n_long_short:]

                    if t + h < len(returns):  # ensure the data not out of range
                        future_h = returns.iloc[t+h][long_h].mean() - returns.iloc[t+h][short_h].mean()
                        holding_returns.append(future_h)
            if len(holding_returns) == H: # ensure the each month has H returns from the strategies
                strategy_returns.iloc[t] = np.mean(holding_returns) if holding_returns else np.nan
    
    return strategy_returns

In [8]:
# get the data for table2 Panel A
# Pivot the data to have each predictor's portLS as a column
table2_LS1 = table2_data_monthly.pivot(columns='predictor', values='portLS') # long top 20%, short bottom 20%
table2_LS2 = table2_data_monthly.pivot(columns='predictor', values='portLS2') # Long top 40%, short bottom 40%

# Pivot the data to have each predictor's portLS as a column for the original data
table2_LS1_original = table2_data_monthly_original.pivot(columns='predictor', values='portLS') # long top 20%, short bottom 20%
table2_LS2_original = table2_data_monthly_original.pivot(columns='predictor', values='portLS2') # Long top 40%, short bottom 40%

# Compute Strategies
N = len(table2_data_monthly['predictor'].unique())  # Fixed number of factors
n = max(round(3/20 * N), 1)  # n = 9

# Long TOP 20% and Short BOTTOM 20% of factors for the update data
# L=1, H=1
LS1_1_1 = factor_momentum_strategy(table2_LS1, L=1, H=1, n_long_short=n)
# L=6, H=6
LS1_6_6 = factor_momentum_strategy(table2_LS1, L=6, H=6, n_long_short=n)

# Long TOP 40% and Short BOTTOM 40% of factors
# L=1, H=1
LS2_1_1 = factor_momentum_strategy(table2_LS2, L=1, H=1, n_long_short=n)
# L=6, H=6
LS2_6_6 = factor_momentum_strategy(table2_LS2, L=6, H=6, n_long_short=n)

# Long TOP 20% and Short BOTTOM 20% of factors for the original data
# L=1, H=1
LS1_1_1_original = factor_momentum_strategy(table2_LS1_original, L=1, H=1, n_long_short=n)
# L=6, H=6
LS1_6_6_original = factor_momentum_strategy(table2_LS1_original, L=6, H=6, n_long_short=n)

# Long TOP 40% and Short BOTTOM 40% of factors for the original data
# L=1, H=1
LS2_1_1_original = factor_momentum_strategy(table2_LS2_original, L=1, H=1, n_long_short=n)
# L=6, H=6
LS2_6_6_original = factor_momentum_strategy(table2_LS2_original, L=6, H=6, n_long_short=n)


In [9]:
def compute_statistics(returns, H):
    returns = returns.dropna()
    
    # Annualized return: Monthly mean * 12
    ann_return = returns.mean() * 12 * 100  # In percentage
    
    # Annualized standard deviation: Monthly std * sqrt(12)
    ann_std = returns.std() * np.sqrt(12) * 100  # In percentage
    
    # Compute Newey-West t-value
    # Fit a constant-only model (equivalent to computing the mean)
    X = np.ones(len(returns))  # Constant term for mean
    model = OLS(returns, X).fit()
    
    # Compute Newey-West standard error
    # Choose lags based on H (e.g., H for overlapping returns, or a rule like floor(0.75 * T^(1/3)))
    lags = 4 if H == 1 else H# lag 4 month if H=1
    nw_cov = cov_hac(model, nlags=lags)  # Newey-West covariance matrix
    nw_se = np.sqrt(nw_cov[0, 0])  # Standard error of the mean (constant term)
    
    # Newey-West t-value: mean / Newey-West standard error
    t_value = returns.mean() / nw_se
    
    return ann_return, ann_std, t_value

In [17]:
# Define the filter_returns function
def filter_returns(returns, start_date, end_date):
    return returns[(returns.index >= start_date) & (returns.index <= end_date)].dropna()

# Define strategies with updated and original data
strategies = {
    'LS1_1_1': {'L': 1, 'H': 1, 'Factor': 'LS1', 'data': LS1_1_1},
    'LS1_6_6': {'L': 6, 'H': 6, 'Factor': 'LS1', 'data': LS1_6_6},
    'LS2_1_1': {'L': 1, 'H': 1, 'Factor': 'LS2', 'data': LS2_1_1},
    'LS2_6_6': {'L': 6, 'H': 6, 'Factor': 'LS2', 'data': LS2_6_6},
    'LS1_1_1_original': {'L': 1, 'H': 1, 'Factor': 'LS1', 'data': LS1_1_1_original},
    'LS1_6_6_original': {'L': 6, 'H': 6, 'Factor': 'LS1', 'data': LS1_6_6_original},
    'LS2_1_1_original': {'L': 1, 'H': 1, 'Factor': 'LS2', 'data': LS2_1_1_original},
    'LS2_6_6_original': {'L': 6, 'H': 6, 'Factor': 'LS2', 'data': LS2_6_6_original}
}

# Define time periods, including the original period
time_periods = [
    ('all', datetime(2000, 1, 1), datetime(2023, 12, 31)),
    ('sample', datetime(2000, 1, 1), datetime(2016, 12, 31)),
    ('post', datetime(2017, 1, 1), datetime(2023, 12, 31)),
    ('original', datetime(1963, 7, 1), datetime(2016, 12, 31))
]

# Compute statistics for each strategy and period
results = []
for period_name, start_date, end_date in time_periods:
    for strategy_name, params in strategies.items():
        L, H, factor, data = params['L'], params['H'], params['Factor'], params['data']
        # Skip if the period is not applicable to the data
        if period_name == 'original' and 'original' not in strategy_name:
            continue
        if period_name in ['all', 'sample', 'post'] and 'original' in strategy_name:
            continue
        
        filtered_returns = filter_returns(data, start_date, end_date)
        ann_return, ann_std, t_value = compute_statistics(filtered_returns, H)
        results.append({
            'Factor': factor,
            'Formation period': L,
            'Holding period': H,
            'Period': period_name,
            'Annualized return': ann_return,
            'Standard deviation': ann_std,
            't-value': t_value
        })

# Create DataFrame and format the results
table2_pa_results = pd.DataFrame(results)
table2_pa_results['Period'] = pd.Categorical(table2_pa_results['Period'], categories=['all', 'sample', 'post', 'original'], ordered=True)
table2_pa_results = table2_pa_results.sort_values(by=['Period', 'Factor', 'Formation period', 'Holding period'])
table2_pa_results = table2_pa_results[['Period', 'Factor', 'Formation period', 'Holding period', 'Annualized return', 'Standard deviation', 't-value']]
table2_pa_results = table2_pa_results.round(2)

# Print the results in the desired format
print("Panel A: Annualized percent returns and standard deviations")
table2_pa_results

Panel A: Annualized percent returns and standard deviations


,Period,Factor,Formation period,Holding period,Annualized return,Standard deviation,t-value
0,all,LS1,1,1,5.64,25.44,1.38
1,all,LS1,6,6,0.76,8.02,0.27
2,all,LS2,1,1,12.12,38.69,1.86
3,all,LS2,6,6,1.82,13.16,0.39
4,sample,LS1,1,1,5.25,27.50,1.09
5,sample,LS1,6,6,0.43,8.58,0.12
6,sample,LS2,1,1,11.46,41.51,1.49
7,sample,LS2,6,6,0.87,14.06,0.15
8,post,LS1,1,1,6.61,19.69,0.87
9,post,LS1,6,6,1.60,6.40,0.47


### Panel B

In [11]:
# Panel B
start_date = '2000-01-01'
end_date = '2023-12-31'
# combine the FF5 and UMD data
table2_ff5 = table2_ff5[(table2_ff5['date'] >= start_date) & (table2_ff5['date'] <= end_date)]
table2_UMD = table2_UMD[(table2_UMD['date'] >= start_date) & (table2_UMD['date'] <= end_date)]
table2_FF5_UMD = pd.merge(table2_ff5, table2_UMD, on='date', how='inner')
table2_FF5_UMD.set_index('date', inplace=True)
table2_FF5_UMD = table2_FF5_UMD[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','UMD']] / 100

# combine the LS data
table2_LS_11 = pd.concat([LS1_1_1, LS2_1_1], axis=1)
table2_LS_11.columns = ['LS1_1_1', 'LS2_1_1']
table2_LS_66 = pd.concat([LS1_6_6, LS2_6_6], axis=1)
table2_LS_66.columns = ['LS1_6_6', 'LS2_6_6']
table2_LS_11.dropna(inplace=True)
table2_LS_66.dropna(inplace=True)
# merge the LS data with FF5 and UMD data
table2_FF5_UMD_LS_11 = pd.merge(table2_FF5_UMD, table2_LS_11, left_index=True, right_index=True, how='inner')
table2_FF5_UMD_LS_66 = pd.merge(table2_FF5_UMD, table2_LS_66, left_index=True, right_index=True, how='inner')

In [12]:
# Define the dependent variables (strategy returns from Panel A)
dependent_vars = {
   'LS1_1_1': table2_FF5_UMD_LS_11 ['LS1_1_1'],  # Standard factor momentum, L=1, H=1
   'LS1_6_6': table2_FF5_UMD_LS_66['LS1_6_6'],  # Standard factor momentum, L=6, H=6
   'LS2_1_1': table2_FF5_UMD_LS_11 ['LS2_1_1'],  # Industry-adjusted factor momentum, L=1, H=1
   'LS2_6_6': table2_FF5_UMD_LS_66['LS2_6_6']   # Industry-adjusted factor momentum, L=6, H=6
}

# Define the independent variables (factors, excluding RF as per factor model convention)
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']

# Function to run spanning regression
def run_spanning_regression(dep_var, ind_vars, label):
    X = sm.add_constant(ind_vars)
    model = sm.OLS(dep_var, X).fit()  
    print(f"\nRegression Results for {label}:")
    print(model.summary())
    return model.params['const'], model.tvalues['const']

In [13]:
# Run regressions for each strategy ( FF5 + UMD only)
results = {}

# 1. Long-Short 20% (L=1, H=1)
ind_vars_1 = table2_FF5_UMD_LS_11[factors]
alpha_1, t_stat_1 = run_spanning_regression(dependent_vars['LS1_1_1'], ind_vars_1, 'Long-Short 20%, L=1, H=1')
results['LS1_1_1'] = (alpha_1, t_stat_1)  # Convert to percent


Regression Results for Long-Short 20%, L=1, H=1:
                            OLS Regression Results                            
Dep. Variable:                LS1_1_1   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     8.066
Date:                Thu, 13 Mar 2025   Prob (F-statistic):           4.76e-08
Time:                        14:26:25   Log-Likelihood:                 366.77
No. Observations:                 288   AIC:                            -719.5
Df Residuals:                     281   BIC:                            -693.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [14]:
# 2. Long-Short 20% (L=6, H=6)
ind_vars_2 = table2_FF5_UMD_LS_66[factors]
alpha_2, t_stat_2 = run_spanning_regression(dependent_vars['LS1_6_6'], ind_vars_2, 'Long-Short 20%, L=6, H=6')
results['LS1_6_6'] = (alpha_2, t_stat_2)


Regression Results for Long-Short 20%, L=6, H=6:
                            OLS Regression Results                            
Dep. Variable:                LS1_6_6   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     3.253
Date:                Thu, 13 Mar 2025   Prob (F-statistic):            0.00417
Time:                        14:26:25   Log-Likelihood:                 674.37
No. Observations:                 283   AIC:                            -1335.
Df Residuals:                     276   BIC:                            -1309.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [15]:
# 3. Long-Short 40% (L=1, H=1)
ind_vars_3 = table2_FF5_UMD_LS_11[factors]
alpha_3, t_stat_3 = run_spanning_regression(dependent_vars['LS2_1_1'], ind_vars_3, 'Long-Short 40%, L=1, H=1')
results['LS2_1_1'] = (alpha_3, t_stat_3)


Regression Results for Long-Short 40%, L=1, H=1:
                            OLS Regression Results                            
Dep. Variable:                LS2_1_1   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     9.303
Date:                Thu, 13 Mar 2025   Prob (F-statistic):           2.62e-09
Time:                        14:26:25   Log-Likelihood:                 249.22
No. Observations:                 288   AIC:                            -484.4
Df Residuals:                     281   BIC:                            -458.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

In [16]:
# 4. Long-Short 40% (L=6, H=6)
ind_vars_4 = table2_FF5_UMD_LS_66[factors]
alpha_4, t_stat_4 = run_spanning_regression(dependent_vars['LS2_6_6'], ind_vars_4, 'Long-Short 40%, L=6, H=6')
results['LS2_6_6'] = (alpha_4, t_stat_4)


Regression Results for Long-Short 40%, L=6, H=6:
                            OLS Regression Results                            
Dep. Variable:                LS2_6_6   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     2.884
Date:                Thu, 13 Mar 2025   Prob (F-statistic):            0.00964
Time:                        14:26:25   Log-Likelihood:                 533.11
No. Observations:                 283   AIC:                            -1052.
Df Residuals:                     276   BIC:                            -1027.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co